In [3]:
import os

base = r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT"
folders = [
    "data/images",
    "data/population",
    "data/roads",
    "data/boundaries",
    "data/processed",
    "data/patches",
    "outputs/maps",
    "notebooks"
]
for f in folders:
    path = os.path.join(base, f)
    os.makedirs(path, exist_ok=True)
    print("Ensured:", path)


Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/images
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/population
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/roads
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/boundaries
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/processed
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data/patches
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\outputs/maps
Ensured: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\notebooks


In [5]:
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
import os

# Reference raster (from your satellite data, ensures same grid/extent)
ref_raster = r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\images\LC09_L2SP_147047_20250902_20250904_02_T2_SR_B3.TIF"

# Input population rasters (WorldPop/GPW for India)
pop_files = {
    2000: r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\population\ind_pd_2000_1km_UNadj.tif",
    2005: r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\population\ind_pd_2005_1km_UNadj.tif",
    2010: r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\population\ind_pd_2010_1km_UNadj.tif",
    2015: r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\population\ind_pd_2015_1km_UNadj.tif",
    2020: r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\population\ind_pd_2020_1km_UNadj.tif"
}

# Output folder
out_dir = r"C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed"
os.makedirs(out_dir, exist_ok=True)

# Load reference metadata (satellite band)
with rasterio.open(ref_raster) as ref:
    ref_meta = ref.meta.copy()

# Loop through all population years
for year, pop_in in pop_files.items():
    pop_out = os.path.join(out_dir, f"resampled_pop_{year}.tif")
    
    with rasterio.open(pop_in) as src:
        src_arr = src.read(1)

        dst = np.empty((ref_meta['height'], ref_meta['width']), dtype='float32')

        reproject(
            source=src_arr,
            destination=dst,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_meta['transform'],
            dst_crs=ref_meta['crs'],
            resampling=Resampling.bilinear
        )

        out_meta = ref_meta.copy()
        out_meta.update(dtype='float32', count=1)

        with rasterio.open(pop_out, 'w', **out_meta) as dstf:
            dstf.write(dst.astype('float32'), 1)

    print(f"✅ Saved resampled population raster: {pop_out}")


✅ Saved resampled population raster: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed\resampled_pop_2000.tif
✅ Saved resampled population raster: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed\resampled_pop_2005.tif
✅ Saved resampled population raster: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed\resampled_pop_2010.tif
✅ Saved resampled population raster: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed\resampled_pop_2015.tif
✅ Saved resampled population raster: C:\Users\Lenovo\Desktop\LY MAJOR PROJECT\data\processed\resampled_pop_2020.tif
